In [ ]:
import os
from pathlib import Path
from joblib import Parallel, delayed
from tqdm import tqdm
import shutil
import itertools
import numpy as np
import time
import math
import cv2
import torch
from torch.utils.data import DataLoader
from utils import FaceInferenceImageDataset, FaceDataset, PyLModel, load_inference_dir

#import matplotlib.pyplot as plt
#%matplotlib inline

raw_dir = Path.cwd() / "data_scale/raw"
input_dir = Path.cwd() / "data_scale/output"
output_dir = Path.cwd() / "data_filter/raw"

In [ ]:
shutil.rmtree(output_dir, ignore_errors=True)
time.sleep(1)
output_dir.parent.mkdir(exist_ok=True)
output_dir.mkdir(exist_ok=True)

In [ ]:
parameters = load_inference_dir(input_dir)
parameters = [(p.name, p.stem.split("_")) for p in parameters]
assert(min([len(l) for _, l in parameters]) >= 4)
parameters = [(p, "_".join(l[0:4]), len(l)-4) for p, l in parameters]
parameters = [(raw_dir / (src_stem + ".png"), output_dir / dst_filename, scale) for dst_filename, src_stem, scale in parameters]
print(max([s for _, _, s in parameters]))

In [ ]:
scale_factor = 0.1
estimation_margin = 2

def process(params):
    src_path, dst_path, scale = params

    scale = max(0, scale - estimation_margin)
    print(src_path)

    image = cv2.imread(str(src_path), cv2.IMREAD_COLOR)
    image_h, image_w, _ = image.shape

    remove_ratio = 1.0 - (1.0 - scale_factor) ** scale
    trim_w = math.ceil(image_w * remove_ratio * 0.5)
    trim_h = math.ceil(image_h * remove_ratio * 0.5)

    crop_image = image[trim_h:image_h-trim_h-1, trim_w:image_w-trim_w-1, :]
    crop_image = cv2.resize(crop_image, (image_w, image_h), interpolation = cv2.INTER_CUBIC)

    if (False):
        plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        plt.show()
        plt.imshow(cv2.cvtColor(crop_image, cv2.COLOR_BGR2RGB))
        plt.show()

    cv2.imwrite(str(dst_path), crop_image)

#[process(params) for params in tqdm(parameters)]
Parallel(n_jobs=-1, verbose=10)([delayed(process)(params) for params in parameters])
print("output done.")